In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
input_data = pd.read_csv("./train.csv")
input_data.head()

In [ ]:
data=np.array(input_data)
print(data)
m,n=data.shape
np.random.shuffle(data)

In [ ]:
data_dev = data[0:1000].T
Xdev=ind[1:n]
ydev=data_dev[0]

data_train=data[1000:m].T
Xtrain=data_train[1:n]
ytrain = data_train[0]
Xtrain.shape
ytrain.shape

In [ ]:
def RELU(x):
    return np.maximum(0,x)

def dRELU(x):
    return x>0

def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1 - np.tanh(x)**2

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [ ]:
def onehot(x):
    ohX=np.zeros((x.size,x.max()+1))
    ohX[np.arange(x.size),x]=1
    return ohX.T

In [ ]:
def initparams(X):
    m,n=X.shape
    W1 = np.random.rand(10, m) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return W1,b1,W2,b2

def fpass(X,W1,b1,W2,b2):
    Z1=np.dot(W1,X)+b1
    A1=tanh(Z1)
    
    Z2 = np.dot(W2,A1)+b2
    A2=softmax(Z2)
    
    return Z1,A1,Z2,A2
    
def bpass(X,y,Z1,A1,Z2,A2,W2):
    m=y.size
    y=onehot(y)
    
    dZ2 = A2 - y
    dW2 = 1 / m * np.dot(dZ2,A1.T)
    db2 = 1 / m * np.sum(dZ2, axis = 1, keepdims = True)
    
    dZ1 = np.dot(W2.T,dZ2) * dtanh(A1)
    dW1 = 1 / m * np.dot(dZ1,X.T)
    db1 = 1 / m * np.sum(dZ1, axis = 1, keepdims = True)
    
    return dW1,db1,dW2,db2

def updateparams(W1,b1,W2,b2,dW1,db1,dW2,db2,lr):
    W1= W1 - lr*dW1
    b1= b1 - lr*db1
    
    W2= W2 - lr*dW2
    b2= b2 - lr*db2
    
    return W1,b1,W2,b2

def predict(x):
    return np.argmax(x,0)

def accuracy(preds,x):
    print(preds,x)
    return np.sum(preds==x)/x.size
    

def train(X,y,itr):
    W1,b1,W2,b2=initparams(X)
    lr=0.08
    
    for i in range(itr+1):
        Z1,A1,Z2,A2=fpass(X,W1,b1,W2,b2)
        dW1,db1,dW2,db2=bpass(X,y,Z1,A1,Z2,A2,W2)
        W1,b1,W2,b2=updateparams(W1,b1,W2,b2,dW1,db1,dW2,db2,lr)
        
        if i%50 ==0:
            print(f"iteration: {i}")
            print(f"accuracy: {accuracy(predict(A2),y)}") 
    
    
    

In [ ]:
train(Xtrain, ytrain, 200)